# 월별 물동량 추이
### 크롤링

In [3]:
import numpy as np
import pandas as pd
from selenium import webdriver #Selenium의 웹 드라이버를 사용하기 위한 모듈을 임포트
from selenium.webdriver.common.by import By #Selenium에서 사용하는 By 클래스를 임포트합. 웹 요소를 검색하는데 사용.
from selenium.webdriver.common.keys import Keys #키보드 입력 제어를 위해 Keys 클래스 임포트
from selenium.webdriver.chrome.service import Service #Chrome 드라이버 서비스를 사용하기 위한 모듈 임포트
from selenium.webdriver.chrome.options import Options #Chrome 드라이버 옵션을 설정하기 위한 클래스 임포트
from webdriver_manager.chrome import ChromeDriverManager #Chrome 드라이버를 자동으로 설치 및 관리하는데 사용되는 드라이버 매니저 임포트
from bs4 import BeautifulSoup #Beautiful soup 크롤링을위해
import time

In [28]:
def get_monthly_shipments():
    # 브라우저 꺼짐 방지
    chrome_options = Options()
    chrome_options.add_experimental_option("detach", True)
    # 불필요한 에러 메시지 없애기
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=chrome_options)
    driver.get('https://www.chainportal.co.kr/portstat/nexacro/index.html?screenid=screen_stat')
    driver.maximize_window()

    driver.implicitly_wait(20)
    # 로그인 진행
    # id 입력창과 pw 입력창 찾기
    id_btn=driver.find_element(By.CSS_SELECTOR,'#mainframe\.frameIndex\.form\.divCenter\.form\.divLogin\.form\.edId\:input')
    pw_btn=driver.find_element(By.CSS_SELECTOR,'#mainframe\.frameIndex\.form\.divCenter\.form\.divLogin\.form\.edPw\:input')
    # id 입력
    id_btn.send_keys('zoqxls98')

    # pw 입력창 클릭 - 안하면 send_keys() 불가능
    pw_btn.click()
    time.sleep(1)
    pw_btn.send_keys('Rlatjddlf527&')
    
    # 로그인 버튼 클릭
    login_btn=driver.find_element(By.CSS_SELECTOR,'#mainframe\.frameIndex\.form\.divCenter\.form\.divLogin\.form\.btnLogin\:icontext')
    login_btn.click()
    print('Log in Done')
    # 사이드 창에서 월별 화물 처리실적 찾아 들어가기
    time.sleep(10)
    btn_1=driver.find_element(By.CSS_SELECTOR,'#mainframe\.frameIndex\.form\.divFrameLeft\.form\.divLeft\.form\.grdTree\.body\.gridrow_2\.cell_2_1\:text')
    btn_1.click()
    time.sleep(2)
    btn_2=driver.find_element(By.CSS_SELECTOR,'#mainframe\.frameIndex\.form\.divFrameLeft\.form\.divLeft\.form\.grdTree\.body\.gridrow_4\.cell_4_1\:text')
    btn_2.click()
    print('Site load Done')

    # 컨테이너 / 일반 구분 체크박스 클릭
    time.sleep(5)
    text_btn=driver.find_elements(By.CLASS_NAME,'nexatextitem')  #find_elements로 class name이 nexatextitem인 요소 모두 찾기
    time.sleep(10)
    divide_btn=text_btn[14] #슬라이싱으로 체크박스만 선택
    divide_btn.click()  #클릭
    print('Dvide btn clicked')
    
    # 연도 입력 후 월별 물동량 크롤링(18년도 ~ 23년)
    # 
    df_list=[]
    year_list=list(range(2018,2024))
    for i in year_list:
        time.sleep(5)
        year_btn=driver.find_elements(By.CLASS_NAME,'nexainput')[6]
        year_btn.click()
        year_btn.clear()
        time.sleep(5)
        year_btn.send_keys(str(i))
        time.sleep(5)
        search_btn=text_btn[13]
        search_btn.click()
        print('Search btn clicked')
        time.sleep(3)
        df=pd.DataFrame()
        month_list_raw=driver.find_elements(By.CLASS_NAME,'nexacontentsbox nexacenteralign')
        time.sleep(10)
        print('월 raw list 개수 : ',len(month_list_raw))
        shipments_list_raw=driver.find_elements(By.CLASS_NAME,'nexacontentsbox nexarightalign')
        time.sleep(10)
        print('물동량 raw list 개수 : ',len(shipments_list_raw))
        month_list=[]
        shipments_list=[]
        time.sleep(20)
        for j in range(16,28):
            month=month_list_raw[i].text
            month_list.append(month)
        print(str(i)+' - months done')
        shipment=[]
        for j in range(12):
            j=96+(j*10)
            shipment=shipments_list_raw[j].text
            shipments_list.append(shipment)
        print(str(i)+' - shipments done')
        df=pd.DataFrame({
            'year':i,
            'month':month_list,
            'shipments':shipments_list
        })
        df_list.append(df)
        time.sleep(5)
        print(str(i)+' done')
    driver.close()
    return df_list

In [29]:
df_list=get_monthly_shipments()

Log in Done
Site load Done
Dvide btn clicked
Search btn clicked
월 raw list 개수 :  0
물동량 raw list 개수 :  0


IndexError: list index out of range